# The Imports

In [1]:
import sys
sys.path.append('/home/decker/lab433')

from synoptic import MODEL, display_grids

from datetime import datetime

import xarray as xr
from metpy.io import GempakGrid
from metpy.units import units
from metpy.plots import ContourPlot, FilledContourPlot, BarbPlot, MapPanel, PanelContainer
import metpy.calc as mpcalc

# Get the Data

In [2]:
gem_file_name = MODEL + 'nam/24092512_nam211.gem'
gem_file = GempakGrid(gem_file_name)

As always, when looking at data for the first time, it is a good idea to see just what it looks like.

In [3]:
list_of_grids = gem_file.gdinfo()

Now we get down to the business of picking out the data we want. What's the time, what level in the atmosphere, what is the parameter called, etc. Because `gdxarray` always returns a list, we use `[0]` to grab the first (and only, at least for this request!) item from the list.

MetPy can do a lot of automatic unit conversions, but the GEMPAK data comes in without units. Let's fix that.

# Calculate Something
In this case, we are just plotting the data as is (aside from unit conversions), so no calculations yet.

# Plot It
## What are we plotting?

We can use `help(ContourPlot)` to learn about the attributes that can be specified in the next cell. Notice that we are requesting an automatic unit conversion here.

In [4]:
cp = ContourPlot()
cp.data = ht500.isel(x=range(20,80))
cp.time = plot_time
cp.contours = range(0, 700, 6)
cp.linecolor = 'mediumvioletred'
cp.linestyle = 'solid'
cp.clabels = True
cp.plot_units = 'dam'

NameError: name 'ht500' is not defined

## Where are we plotting?

In [ ]:
panel = MapPanel()
panel.area = [-120, -74, 22, 55]
panel.projection = 'lcc'
panel.layers = ['states', 'coastline', 'borders']
panel.title = f'500-mb Heights at {plot_time}'
panel.plots = [cp]

## Display the Plot

In [ ]:
pc = PanelContainer()
pc.size = (15, 15)
pc.panels = [panel]
pc.show()

# Example 2: Filled Contours
Now we will show an example of plotting filled contours, using the 500-mb temperature. This example also shows how we can overlay additional contours on top.

In [ ]:
t500 = gem_file.gdxarray(parameter='TMPK', date_time=plot_time, level=500)[0]

In [ ]:
t500 = t500 * units('degK')

In [ ]:
fill = FilledContourPlot()
fill.data = t500
fill.time = plot_time
fill.contours = range(-20, 10, 5)
fill.colormap = 'twilight'
fill.colorbar = 'horizontal'
fill.plot_units = 'degC'

Although we previously constructed a `ContourPlot` object in the first example, that object was "consumed" when we made the previous plot with `pc.show()`. We need to recreate it from scratch.

In [ ]:
cp = ContourPlot()
cp.data = ht500
cp.time = plot_time
cp.contours = range(460, 700, 6)
cp.linecolor = 'lime'
cp.linestyle = 'solid'
cp.clabels = True
cp.plot_units = 'dam'

In [ ]:
panel = MapPanel()
panel.area = [-120, -74, 22, 55]
panel.projection = 'lcc'
panel.layers = ['states', 'coastline', 'borders']
panel.title = f'500-mb Heights and Temperatures at {plot_time}'
panel.plots = [fill, cp]

In [ ]:
pc = PanelContainer()
pc.size = (15, 15)
pc.panels = [panel]
pc.show()

# Example 3: Wind Barbs
This time we will add wind barbs to the plot.

In [5]:
u500 = gem_file.gdxarray(parameter='UREL', date_time=plot_time, level=500)[0]
v500 = gem_file.gdxarray(parameter='VREL', date_time=plot_time, level=500)[0]
u500 = u500 * units('m/s')
v500 = v500 * units('m/s')

ht1000 = gem_file.gdxarray(parameter='HGHT', date_time=plot_time, level=1000)[0] *units ('m')
ht500 = gem_file.gdxarray(parameter='HGHT', date_time=plot_time, level=500)[0] *units ('m')

ht500 = mpcalc.smooth_gaussian(ht500, 12)
ht1000 = mpcalc.smooth_gaussian(ht1000, 12)

ug500, vg500 = mpcalc.geostrophic_wind(ht500)
ug1000, vg1000 = mpcalc.geostrophic_wind(ht1000)

vt_i = ug500.squeeze(dim='pres') - ug1000.squeeze(dim='pres')
vt_j = vg500.squeeze(dim='pres') - vg1000.squeeze(dim='pres')

zetag_plus_f = mpcalc.absolute_vorticity(ug500, vg500)
zetag_zero = mpcalc.vorticity(ug1000, vg1000)

term_in_parenthesis = (zetag_plus_f.squeeze(dim='pres') + zetag_zero.squeeze(dim='pres'))

answer = mpcalc.advection(term_in_parenthesis, vt_i, vt_j)

NameError: name 'plot_time' is not defined

In [6]:
wind = xr.merge([u500, v500])
wind

NameError: name 'u500' is not defined

In [ ]:
barbs = BarbPlot()
barbs.data = wind
barbs.time = plot_time
barbs.field = ['urel', 'vrel']
barbs.earth_relative = False
barbs.skip = (3, 3)
barbs.plot_units = 'knot'

In [ ]:
cp = ContourPlot()
cp.data = answer
cp.time = plot_time
cp.contours = range(0, 10, 1)
cp.linecolor = 'mediumturquoise'
cp.linestyle = 'solid'
cp.clabels = True
cp.scale = 1e9

In [ ]:
panel = MapPanel()
panel.area = [-120, -74, 22, 55]
panel.projection = 'lcc'
panel.layers = ['states', 'coastline', 'borders']
panel.title = f'500-mb Heights and Winds at {plot_time}'
panel.plots = [cp]

In [ ]:
pc = PanelContainer()
pc.size = (15, 15)
pc.panels = [panel]
pc.show()